# Google Fiber: Data Cleaning

## Required Data Fields
- **Number of calls**
- **Number of repeat calls after first contact**
- **Call type**:
  - `Type_1` = Account management
  - `Type_2` = Technician troubleshooting
  - `Type_3` = Scheduling
  - `Type_4` = Construction
  - `Type_5` = Internet & Wi-Fi
- **Market city**: `market_1`, `market_2`, `market_3` represent three anonymized city service areas.
- **Date**: `contacts_n` = initial contact date; subsequent columns like `contacts_n_6` indicate number of days since first contact (e.g., 6 days after initial contact).

In [1]:
import pandas as pd

market_1, market_2, market_3 = (pd.read_csv('raw-data/market_1.csv'),
                                pd.read_csv('raw-data/market_2.csv'),
                                pd.read_csv('raw-data/market_3.csv'))

# Concatenate the datasets vertically
df = pd.concat([market_1, market_2, market_3], axis=0)

df.info()  # Check data types to ensure every column has the correct format
print(f"\nTotal duplicated records: {df.duplicated().sum()}")  # Count duplicate rows; none expected in this dataset

<class 'pandas.core.frame.DataFrame'>
Index: 1350 entries, 0 to 449
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date_created  1350 non-null   object 
 1   contacts_n    1168 non-null   float64
 2   contacts_n_1  892 non-null    float64
 3   contacts_n_2  821 non-null    float64
 4   contacts_n_3  778 non-null    float64
 5   contacts_n_4  739 non-null    float64
 6   contacts_n_5  719 non-null    float64
 7   contacts_n_6  716 non-null    float64
 8   contacts_n_7  733 non-null    float64
 9   new_type      1350 non-null   object 
 10  new_market    1350 non-null   object 
dtypes: float64(8), object(3)
memory usage: 126.6+ KB

Total duplicated records: 0


In [36]:
# Calculate the total number of repeat calls across days 1 to 7
df['Total Repeat Calls'] = df[['contacts_n_1','contacts_n_2','contacts_n_3',
                               'contacts_n_4','contacts_n_5','contacts_n_6','contacts_n_7']].sum(axis=1)

# Rename columns to improve readability for analysis and visualization
df = df.rename(columns={
    'date_created': 'Date',
    'contacts_n': 'Calls',
    'contacts_n_1': 'Day 1',
    'contacts_n_2': 'Day 2',
    'contacts_n_3': 'Day 3',
    'contacts_n_4': 'Day 4',
    'contacts_n_5': 'Day 5',
    'contacts_n_6': 'Day 6',
    'contacts_n_7': 'Day 7',
    'new_type': 'Type',
    'new_market': 'Market',
})

# Rename specific problem types for readability
df['Type'] = df['Type'].replace({
    'type_1': 'Account management',
    'type_2': 'Technician troubleshooting',
    'type_3': 'Scheduling',
    'type_4': 'Construction',
    'type_5': 'Internet & Wi-Fi'
})

# Rename specific market types for readability
df['Market'] = df['Market'].replace({
    'market_1': 'Market 1',
    'market_2': 'Market 2',
    'market_3': 'Market 3'
})

In [37]:
# Replace all NaN values with 0 to indicate "no calls" and ensure consistency
df = df.fillna(0)

# Convert float columns to integers, since call counts should be whole numbers
float_cols = df.select_dtypes(include='float').columns
df[float_cols] = df[float_cols].astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1350 entries, 0 to 449
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Date                1350 non-null   object
 1   Calls               1350 non-null   int32 
 2   Day 1               1350 non-null   int32 
 3   Day 2               1350 non-null   int32 
 4   Day 3               1350 non-null   int32 
 5   Day 4               1350 non-null   int32 
 6   Day 5               1350 non-null   int32 
 7   Day 6               1350 non-null   int32 
 8   Day 7               1350 non-null   int32 
 9   Type                1350 non-null   object
 10  Market              1350 non-null   object
 11  Total Repeat Calls  1350 non-null   int32 
dtypes: int32(9), object(3)
memory usage: 89.6+ KB


In [38]:
# Split the 'Date' column into separate 'Day', 'Month', and 'Year' columns
df['Date'] = pd.to_datetime(df['Date'])

df['Day'] = df['Date'].dt.day         # Split each date value in separate columns
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

# Format the datetime column as DD-MM-YYYY
df['Date'] = df['Date'].dt.strftime('%d-%m-%Y')

In [39]:
# Reorder the columns into a logical sequence for readability
new_order = ['Date','Day', 'Month', 'Year', 'Calls', 'Day 1', 'Day 2', 'Day 3',
             'Day 4', 'Day 5', 'Day 6', 'Day 7', 'Total Repeat Calls', 'Type', 'Market']

df = df[new_order]

# Sample of the final dataset
df.head()

,Date,Day,Month,Year,Calls,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7,Total Repeat Calls,Type,Market
0,04-02-2022,4,2,2022,199,21,6,11,7,14,5,6,70,Internet & Wi-Fi,Market 1
1,30-01-2022,30,1,2022,19,2,0,2,1,0,0,0,5,Account management,Market 1
2,14-02-2022,14,2,2022,29,0,2,2,0,1,0,1,6,Account management,Market 1
3,16-01-2022,16,1,2022,120,6,6,5,4,7,4,0,32,Technician troubleshooting,Market 1
4,03-02-2022,3,2,2022,182,27,13,0,14,4,3,2,63,Internet & Wi-Fi,Market 1


In [40]:
# Export the combined and cleaned dataset
df.to_csv('data.csv', index=False)